In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

from functools import partial

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score


In [ ]:
diamond=pd.read_csv('diamonds_train.csv')
diamond.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95


In [ ]:
X=diamond.drop(columns=['price'])
y=diamond.price

In [ ]:
clarity={'I1':0, 'SI2':1, 'SI1':2, 'VS2':3, 'VS1':4, 'VVS2':5, 'VVS1':6, 'IF':7}
cut={'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}
color={'J':0, 'I':1, 'H':2, 'G':3, 'F':4, 'E':5, 'D':6}

In [ ]:
def labeling(s, dic):
    return dic[s]

In [ ]:
X.clarity=X.clarity.apply(lambda x: labeling(x, clarity))
X.cut=X.cut.apply(lambda x: labeling(x, cut))
X.color=X.color.apply(lambda x: labeling(x, color))

In [ ]:
X.head()

In [ ]:
X=X.drop(columns=['table'])
X.head()

In [ ]:
X['Vol']=X.x*X.y*X.z
X['Sum']=X.carat**2+2*X.clarity+X.color
X.head()
X[X.Vol==0]

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
gbm = lgb.LGBMRegressor(num_leaves=100,
                        learning_rate=0.05,
                        n_estimators=200)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2',
        early_stopping_rounds=50, verbose=False)


In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print('RMSE:', mean_squared_error(y_test, y_pred)**0.5)

# Bayes-gridsearch (hyperopt)

In [ ]:
hp_space = {
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart']), 
    'learning_rate': hp.uniform('learning_rate', 0.0001, 1.0),
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'n_estimators': hp.quniform('n_estimators', 25, 1000, 25),
    'reg_alpha':hp.uniform('reg_alpha', 0.1, 1.0)
}


ho_sample(hp_space)

In [ ]:
space ={'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.5),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }

In [ ]:
def objetivo(space):
    #print(space)
    clf = xgb.XGBRegressor(n_estimators =10,
                           learning_rate = .3,
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],)

    eval_set=[(X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric="rmse", verbose=False)

    y_pred = clf.predict(X_train)
    rmse = mean_squared_error(y_train, y_pred)**(0.5)
    #print ("SCORE:", np.sqrt(mse_scr))
    #change the metric if you like
    return {'loss':rmse, 'status': STATUS_OK }

In [ ]:
trials_reg = Trials()
best = fmin(fn=objetivo,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials_reg)

In [ ]:
print (best)

In [ ]:
modelo=xgb.XGBRegressor(x_gamma=0.3050325965629337, x_max_depth= 13.0, x_min_child= 4.0, 
                        x_reg_lambda=0.1549679872678349, x_subsample= 0.9002594146971687)

# Submit

In [ ]:
def submit(modelo, X, y):
    
    clarity={'I1':0, 'SI2':1, 'SI1':2, 'VS2':3, 'VS1':4, 'VVS2':5, 'VVS1':6, 'IF':7}
    cut={'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}
    color={'J':0, 'I':1, 'H':2, 'G':3, 'F':4, 'E':5, 'D':6}
    
    df=pd.read_csv('diamonds_test.csv')
    submit=pd.DataFrame()
    submit['id']=df.id
    df=df.drop(columns='id')
    
    df.clarity=df.clarity.apply(lambda x: labeling(x, clarity))
    df.cut=df.cut.apply(lambda x: labeling(x, cut))
    df.color=df.color.apply(lambda x: labeling(x, color))
    df=df.drop(columns=['table'])
    df['Vol']=df.x*df.y*df.z
    df['Sum']=df.carat**2+2*df.clarity+df.color

    
    modelo.fit(X, y)
    y_pred=modelo.predict(df)
    
    submit['price']=y_pred
    return submit

In [ ]:
res=submit(modelo, X, y)

In [ ]:
res.head()

In [ ]:
#res.to_csv('submit_lgbm_bayes.csv', index=False)